# Prefix Sanity Check
This report checks for sanity of AFRINIC "Available" and "Reserved" resources. We gather data about the free pool from the AFRINIC delegated file and run check against the CIDR-Report (https://www.cidr-report.org) for Bogons and IRRExplorer (http://irrexplorer.nlnog.net/) for route objects appearing in any IRR.

In [15]:
import numpy as np
import pandas as pd
import urllib, urllib2, json, re
from netaddr import *
from bs4 import BeautifulSoup
from StringIO import StringIO
from IPython.display import HTML

BGP_LOOKING_GLASS_URL="https://stat.ripe.net/data/looking-glass/data.json?resource="
URL_IRREXPLORER = "http://irrexplorer.nlnog.net/json/prefix/"
IPSETS_PATH = "data/ipsets/subset/"
IPRESOURCES_PATH = "ftp://ftp.afrinic.net/stats/afrinic/delegated-afrinic-extended-latest"
V4_BOGON_URL = "https://www.cidr-report.org/as2.0/indext.html"
V6_BOGON_URL = "http://www.cidr-report.org/v6/as2.0/indext.html"
BOGUS_AS_URL = "https://www.cidr-report.org/as2.0/bogus-as-advertisements.html"

def fetchBogons(url):
    try:
        res = urllib2.urlopen(url)
    except Exception, e:
        print e
    
    #res = open(url, "r")

    soup = BeautifulSoup(res, "html.parser")
    pre = soup.findAll('pre')
    
    BOGON_AS = StringIO(pre[12].text)
    
    df_bogon_as = pd.read_csv(BOGON_AS, sep='\s{2,20}', 
                     names=['bogus_as','announced_by', 'announcing_as', 'description'], 
                     skipinitialspace=True, skiprows=0, engine='python')
    
    BOGON_PREFIXES = StringIO(pre[11].text)
    
    df_bogon_prefixes = pd.read_csv(BOGON_PREFIXES, sep='\s{2,20}', 
                     names=['prefix','origin_as', 'description'], 
                     skipinitialspace=True, skiprows=0, engine='python')
    
    return df_bogon_as, df_bogon_prefixes


def findIRRObjects(prefix):
    url = URL_IRREXPLORER + str(prefix)
    
    try:
        response = urllib2.urlopen(url)
        data = json.loads(response.read())
    except urllib2.HTTPError:
        data = "{}"
    
    return data

def getIPRange(prefix, prefixlength):
    
    #check for IPv6
    if ":" in prefix:
        return IPNetwork(prefix + "/" + str(prefixlength))
    else: 
        startip = IPAddress(prefix)
        endipint = int(startip) + int(prefixlength) -1
        endip = IPAddress(endipint)
        range = IPRange(startip, endip)
        return range.cidrs()


#Does the match and returns a list of bogon ASNs
def findBogonASN(df_bogus_as, df_asn):
    
    df_asn_free = getFreeASNSpace(df_asn)

    #df_bogus.iloc[:,[2,3,4]]
    df_bogus_as = df_bogus_as.iloc[1:]

    bogon_asns = df_bogus_as.iloc[:,0].drop_duplicates()

    df_as_found = pd.DataFrame(columns=['asnum', 'status'])

    for index, row in df_asn_free.iterrows():
        for r in bogon_asns:
            search = re.search('AS(.*)', r)
            if search:
                asnum = search.group(1)

            if int(asnum) == int(row['resource']):
                 df_as_found = df_as_found.append({'asnum': "AS"+ row['resource'], 'status': row['status']}, ignore_index=True)
                #print row['resource'] + "," + row['status']
            
    return df_as_found


#Does the match and returns a list of bogon prefixes
def findBogonPrefixes(df_ip, df_bogus_prefixes):
    df_ip_free = getFreeIPSpace(df_ip)

    bogus_prefixes = df_bogus_prefixes.iloc[:,0]

    #df_ip_free = df_ip_free.iloc[:,[2,3]]

    df_found = pd.DataFrame(columns=['prefix', 'origin_as', 'description', 'status'])

    for index, prefix in df_ip_free.iterrows():

        ip_range = getIPRange(prefix['resource'], prefix['prefixlen'])
            
        for index, row in df_bogus_prefixes.iterrows():

            try:
                ip = row['prefix'].encode('utf-8').strip()
                bogon_ip = IPNetwork(ip)
            except AttributeError:
                ip = "None"
            except AddrFormatError:
                continue

            try:
                origin_as = row['origin_as'].encode('utf-8').strip()
            except AttributeError:
                origin_as = "None"

            try:
                description = row['description'].encode('utf-8').strip()
            except AttributeError:
                description = "None"

            intersect_range = IPSet(bogon_ip) & IPSet(ip_range)
            
            if intersect_range.size > 0:
                df_found = df_found.append({'prefix': str(ip), 
                                            'origin_as': origin_as, 'description': description, 
                                            'status': prefix['status']}, ignore_index=True)
                #print ip + "|" + origin_as + "|" + description

    return df_found

def checkIRRs(df_ip):
    df_found = pd.DataFrame(columns=['prefix', 'irr_found', 'status'])
    for index, prefix in df_ip.iterrows():
        ip_range = getIPRange(prefix['resource'], prefix['prefixlen'])
        
        ip = str(ip_range[0])
        if ":" in str(prefix['resource']):
                ip = ip + "/" + str(prefix['prefixlen'])
        
        found = findIRRObjects(ip)
                
        if len(found) > 0:
            
            df_found = df_found.append({'prefix': ip, 
                                        'irr_found': str(len(found)) , 'status': prefix['status']}, ignore_index=True)
            print ip + "|" + str(len(found))
    return df_found

def getFreeIPSpace(df_ip):
    df_ip_available = df_ip.loc[df_ip['status']=='available']
    df_ip_reserved = df_ip.loc[df_ip['status']=='reserved']
    df_ip_free = [df_ip_reserved, df_ip_available]
    df_ip_free
    return pd.concat(df_ip_free)

def getFreeASNSpace(df_asn):
    df_asn_reserved = df_asn.loc[df['status']=='reserved']
    df_asn_available = df_asn.loc[df['status']=='available']
    df_asn_free = [df_asn_reserved, df_asn_available]
    return pd.concat(df_asn_free)

In [10]:
#df_bogus_v4_as, df_bogus_v4_prefixes = fetchBogons("data/CIDRReport.htm")
#df_bogus_v6_as, df_bogus_v6_prefixes = fetchBogons("data/CIDRReportV6.htm")

df_bogus_v4_as, df_bogus_v4_prefixes = fetchBogons(V4_BOGON_URL)
df_bogus_v6_as, df_bogus_v6_prefixes = fetchBogons(V6_BOGON_URL)

df = pd.read_csv(IPRESOURCES_PATH, sep='|', 
                     skiprows=5, keep_default_na=0, 
                     names = ['rir','cc','type','resource','prefixlen','allocdate','status','opaqueid'])

#filter out unnecessary columns
df = df.iloc[:,[1,2,3,4,5,6]]

#a dataframe per type
df_asn = df.loc[df['type']=='asn']
df_ipv4 = df.loc[df['type']=='ipv4']
df_ipv6 = df.loc[df['type']=='ipv6']

In [ ]:
#Get bogon ASNs and bogon prefixes for both v4 and v6
df_bogon_as_v4 = findBogonASN(df_bogus_v4_as, df_asn)
df_bogon_prefix_v4 = findBogonPrefixes(df_ipv4, df_bogus_v4_prefixes)
df_bogon_as_v6 = findBogonASN(df_bogus_v6_as, df_asn)
df_bogon_prefix_v6 = findBogonPrefixes(df_ipv6, df_bogus_v6_prefixes)

## Report on Bogus ASNs and Prefixes

### List of Bogon IPv4 prefixes appearing in the global routing table
We use the CIDR Report https://www.cidr-report.org/as2.0/#Bogons to retrieve bogus IPv4 prefixes (Reserved and Available) in the AFRINIC delegated-stats file. Those prefixes appear in the IPv4 routing table.

In [46]:
HTML(pd.DataFrame(df_bogon_prefix_v4).to_html())

,prefix,origin_as,description,status
0,41.76.136.0/22,"AS37500 -Reserved AS-, ZZ",None,reserved
1,41.76.136.0/24,"AS37500 -Reserved AS-, ZZ",None,reserved
2,41.76.138.0/24,"AS37500 -Reserved AS-, ZZ",None,reserved
3,41.76.139.0/24,"AS37500 -Reserved AS-, ZZ",None,reserved
4,41.76.140.0/22,"AS37500 -Reserved AS-, ZZ",None,reserved
5,41.76.140.0/24,"AS37500 -Reserved AS-, ZZ",None,reserved
6,41.76.141.0/24,"AS37500 -Reserved AS-, ZZ",None,reserved
7,41.76.142.0/24,"AS37500 -Reserved AS-, ZZ",None,reserved
8,41.76.143.0/24,"AS37500 -Reserved AS-, ZZ",None,reserved
9,41.78.92.0/22,"AS14988 BTC-GATE1, BW",None,reserved


### List of Bogon AS appearing in the IPv4 global routing table
Reserved and available ASNs that appear in the BGP Global IPv4 routing table 

In [47]:
HTML(pd.DataFrame(df_bogon_as_v4).to_html())

,asnum,status
0,AS36886,reserved
1,AS37265,reserved
2,AS37330,reserved
3,AS37500,reserved
4,AS16547,available


### List of Bogon IPv6 prefixes appearing in the global routing table
We use the CIDR Report https://www.cidr-report.org/as2.0/#Bogons to retrieve bogus IPv4 prefixes (Reserved and Available) in the AFRINIC delegated-stats file. Those prefixes appear in the IPv4 routing table.

In [48]:
HTML(pd.DataFrame(df_bogon_prefix_v6).to_html())

,prefix,origin_as,description,status
0,2c0f:f590::/32,"AS36974 AFNET-AS, CI",None,reserved


### List of Bogon AS appearing in the IPv6 global routing table
Reserved and available ASNs that appear in the BGP Global IPv6 routing table 

In [ ]:
HTML(pd.DataFrame(df_bogon_as_v6).to_html())

,asnum,status


### Reserved and Available IPv4 space appearing in the IRRs

In [7]:
#Checking IRRs
df_irr_v4 = checkIRRs(getFreeIPSpace(df_ipv4))
HTML(pd.DataFrame(df_irr_v4).to_html())

,prefix,irr_found,status
0,41.75.32.0/20,1,reserved
1,41.76.136.0/21,11,reserved
2,41.78.92.0/22,1,reserved
3,41.78.104.0/22,3,reserved
4,41.78.144.0/22,1,reserved
5,41.78.152.0/22,5,reserved
6,41.79.12.0/22,3,reserved
7,41.79.112.0/22,1,reserved
8,41.139.64.0/18,28,reserved
9,41.190.160.0/19,1,reserved


### Reserved and Available IPv6 space appearing in the IRRs

In [16]:
df_irr_v6 = checkIRRs(getFreeIPSpace(df_ipv6))
HTML(pd.DataFrame(df_irr_v6).to_html())

2001:43f8:1f6::/47|1
2c0f:f590::/29|5
2001:4208::/29|1
2001:42f8::/29|1
2001:43f8:f0::/44|1
2001:43f8:170::/44|1
2001:43f8:350::/44|1
2001:43f8:b00::/44|1
2c00::/13|2
2c0f:fd70::/29|1
2c0f:fe48::/29|3


,prefix,irr_found,status
0,2001:43f8:1f6::/47,1,reserved
1,2c0f:f590::/29,5,reserved
2,2001:4208::/29,1,available
3,2001:42f8::/29,1,available
4,2001:43f8:f0::/44,1,available
5,2001:43f8:170::/44,1,available
6,2001:43f8:350::/44,1,available
7,2001:43f8:b00::/44,1,available
8,2c00::/13,2,available
9,2c0f:fd70::/29,1,available
